#Table of Contents


## 1 Импорт данных и библиотек

In [1]:
#!pip install deeppavlov
#!pip install numpy#==1.19.5
#!pip install tensorflow==1.14.0
#!pip install fasttext
#!python -m deeppavlov install squad_bert
#!pip install torch
#!pip install transformers
#!pip install tokenizers
#!pip install simpletransformers
#!pip install pymorphy2
#!pip install pyyaml==5.4.1

In [ ]:
import numpy as np
import pandas as pd
import  re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.display import display,  clear_output
import ipywidgets as widgets
import collections as c
import pymorphy2
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import torch, transformers, tokenizers
from sklearn.tree import DecisionTreeClassifier
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import warnings
warnings.filterwarnings("ignore")
import torch, tokenizers
import deeppavlov
import os
from pathlib import Path
from deeppavlov import build_model,configs, evaluate_model
from deeppavlov.download import deep_download
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
torch.cuda.empty_cache()
torch.cuda.is_available()

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:

df=pd.read_csv("/content/wiki_movie_plots_deduped.csv")
rev=pd.read_csv("/content/IMDB Dataset.csv")
rev_ru=pd.read_csv('/content/data.csv',error_bad_lines=False)


## 2 EDA

In [ ]:
def welcome (df):
    '''Первичная информация о датафрейме'''
    print ('------------- Первые 5 строк ------------')
    display(df.head())
    print('')
    print('')
    print ('------------- Типы данных ------------')
    print (df.info())
    print('')
    print('')
    print ('------------- Пропуски ------------')
    count = 0
    for element in df.columns:
        if df[element].isna().sum() > 0:
            print(element, ' - ', df[element].isna().sum(), 'пропусков')
            count = +1
        if count == 0:
            print('Пропусков НЕТ')
    print('')
    print('')
    display(pd.DataFrame(round(df.isna().mean()*100)).style.background_gradient('coolwarm'))

In [ ]:
welcome (df)

------------- Первые 5 строк ------------


,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


INFO:numexpr.utils:NumExpr defaulting to 2 threads.




------------- Типы данных ------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34886 entries, 0 to 34885
Data columns (total 8 columns):
Release Year        34886 non-null int64
Title               34886 non-null object
Origin/Ethnicity    34886 non-null object
Director            34886 non-null object
Cast                33464 non-null object
Genre               34886 non-null object
Wiki Page           34886 non-null object
Plot                34886 non-null object
dtypes: int64(1), object(7)
memory usage: 2.1+ MB
None


------------- Пропуски ------------
Пропусков НЕТ
Пропусков НЕТ
Пропусков НЕТ
Пропусков НЕТ
Cast  -  1422 пропусков




,0
Release Year,0
Title,0
Origin/Ethnicity,0
Director,0
Cast,4
Genre,0
Wiki Page,0
Plot,0


## 3 Преобразование данных

In [ ]:
df.dropna(inplace=True)
gen_un=df[df['Genre']=='unknown']
dir_un=df[df['Director']=='Unknown']
df=df[df['Genre']!='unknown']
df=df[df['Director']!='Unknown']
df["Genre"].replace({"sci-fi": "science fiction", "romantic comedy": "romance"}, inplace=True)

In [ ]:
def mapp (val):
    v= c.Counter(df[val]).most_common(20)
    a=list(map(lambda x:x[0],v))
    b=list(map(lambda x:x[1],v))
    return pd.DataFrame({'col1':a,'col2':b})

In [ ]:
def graf (val):
    fig = px.histogram(mapp(val), x='col1',y='col2')
    fig.show()

In [ ]:
def hip (but,val):
    button = widgets.Button(description=but)
    output = widgets.Output()

    display(button, output)

    def on_clicked(b):
        with output:          
            clear_output()
            print('График: ', but)
            display(graf (val))
            
    button.on_click(on_clicked)    

## 4 Графики 

In [ ]:
hip('Жанры','Genre')

Button(description='Жанры', style=ButtonStyle())

Output()

In [ ]:
hip('Режисер','Director') 

Button(description='Режисер', style=ButtonStyle())

Output()

In [ ]:
hip('Национальная принадлежность','Origin/Ethnicity')

Button(description='Национальная принадлежность', style=ButtonStyle())

Output()

In [ ]:
hip('Год выпуска','Release Year')

Button(description='Год выпуска', style=ButtonStyle())

Output()

##  5 Sentiment analysis

In [ ]:
welcome (rev)

------------- Первые 5 строк ------------


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive




------------- Типы данных ------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
review       50000 non-null object
sentiment    50000 non-null object
dtypes: object(2)
memory usage: 781.4+ KB
None


------------- Пропуски ------------
Пропусков НЕТ
Пропусков НЕТ




,0
review,0
sentiment,0


In [ ]:
rev['review'] = rev['review'].str.replace('/','').str.replace('<','').str.replace('>','')
rus_pat='[^А-яа-я0-9]+'
eng_pat='[^A-Za-z0-9]+'

In [ ]:
def remove_trash(list,pat): 
    pattern = eng_pat
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 
      print (list)
    except Exception as e:
      print(e)
    return list

In [ ]:
rev['text_clean'] = remove_trash(rev['review'],eng_pat)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
morph = pymorphy2.MorphAnalyzer()
# функция лемматизации
def lemmatize(row):
    t = []
    text = row['text_clean']
    for word in text.split():
        if len(word)<=3:
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)


INFO:pymorphy2.opencorpora_dict.wrapper:Loading dictionaries from /usr/local/lib/python3.7/dist-packages/pymorphy2_dicts/data
INFO:pymorphy2.opencorpora_dict.wrapper:format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [ ]:
rev['text_clean'] = rev.apply(lemmatize,axis=1)
stops = set(stopwords.words('english'))

In [ ]:
text_transformer = TfidfVectorizer(stop_words=stops, ngram_range=(1,1), lowercase=True, max_features=15000 )

In [ ]:
text = text_transformer.fit_transform(rev['text_clean'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(text, rev['sentiment'], test_size=0.3, random_state=13)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
pred = clf.predict(X_test)
print('Качество модели по метрике F1', f1_score(y_test,pred,average='macro'))

Качество модели по метрике F1 0.718733182074289


In [ ]:
def texts (val,pred,col1,col2):

    old_sent = val[col1]

    predicted_sent = pred
    text=val[col2]

    print(f'\nOld Sentiment:'.ljust(16,' '), f'{[old_sent]}\n')
    print(f'Predicted Sentiment: {predicted_sent}\n')
    print(f'Plot: {text}\n')
    print("-------------------------------------------")

In [ ]:
i=np.random.randint(100)
texts(rev.iloc[i],pred[i],'sentiment','review')


Old Sentiment:  ['positive']

Predicted Sentiment: positive

Plot: "The Cell" is an exotic masterpiece, a dizzying trip into not only the vast mind of a serial killer, but also into one of a very talented director. This is conclusive evidence of what can be achieved if human beings unleash their uninhibited imaginations. This is boldness at work, pushing aside thoughts to fall into formulas and cliches and creating something truly magnificent. This is the best movie of the year to date.br br I've read numerous complaints about this film, anywhere from all style and no substance to poorly cast characters and bad acting. To negatively criticize this film is to miss the point. This movie may be a landmark, a tradition where future movies will hopefully follow. "The Cell" has just opened the door to another world of imagination. So can we slam the door in its face and tell it and its director Tarsem Singh that we don't want any more? Personally, I would more than welcome another movie by 

## 6 Анализ тональности текста

In [ ]:
welcome(rev_ru)

------------- Первые 5 строк ------------


,Unnamed: 0,review,sentiment
0,0,"Уж не знаю, что полезного для себя могут почер...",neg
1,1,"Это моя первая рецензия, обычно я захожу на Ки...",pos
2,2,"Смотрел год и тему, тему и год. В 60-х выпуска...",pos
3,3,"«До свидания, мальчики» - полузабытая лента М....",pos
4,4,Первая крупная роль Эвана МакГрегора – роль Ал...,pos




------------- Типы данных ------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19751 entries, 0 to 19750
Data columns (total 3 columns):
Unnamed: 0    19751 non-null int64
review        19751 non-null object
sentiment     19751 non-null object
dtypes: int64(1), object(2)
memory usage: 463.0+ KB
None


------------- Пропуски ------------
Пропусков НЕТ
Пропусков НЕТ
Пропусков НЕТ




,0
Unnamed: 0,0
review,0
sentiment,0


In [ ]:
rev_ru['sentiment']=rev_ru['sentiment'].apply(lambda x: 'positive' if x=='pos' else( 'negative' if x=='neg' else 'neutral'))

In [ ]:
rev_ru['text_clean']=remove_trash(rev_ru['review'],rus_pat)

[' 5 10', ' 10 10', ' 60 60 ', ' 60 ', ' ', ' 1991 8 prouste 6 10', ' 10 10', ' 1997 1999 8 8 45 100 9 16 9 10', ' 7 ', 'Well generation gap ', ' P S 6 5 10', ' 11 11 11 10 4 10', ' 87 8 10', ' 80 2 80 80 1408 80 34 11 IMDb 4 5 80 16 Zhiliton 2018 20 12 2 ', ' ', ' 2 7 5 10', ' 10 10', ' 5 10', ' Fox Marvel DC 5 10', ' 5 2 6 10', ' 100 4 5 10', ' 7 10', ' B 2012 80 6 10', ' ', ' 1988 ', ' ', ' 2001 2 10', ' 1 2 3 4 5 6 7 8 9 10 10', ' ', ' 1930 1930 ', ' The Master 2013 The Master The Master ', ' 78 4 44 2 10', ' cherchez la femme ', '2009 2009 10 10', ' ', ' 60 20 4 1968 9 5 10', ' 1986 9 10', ' 90 10 10', ' 80 15 2 15 1 6 2 10', ' ', ' 7 10', ' 4 10', ' 10 10 1 45 2 II 150 2 3 10 10 ', ' 8 10', ' dvd i They fought in a brutal war against a system that killed millions and continued to live love and remained unconquered i 1944 1947 i 3 10', ' 29 10 10', ' 7 Haeundae 9 10 ', ' 3 4 90 10 7 10', ' Wayward Pines Twin Peaks 90 2 5 5 10', ' 4 9 10', ' 70 70 77 1984 6 4 4 1 5 8 10', ' Marvel 

In [ ]:
rev_ru['text_clean'] = rev_ru.apply(lemmatize,axis=1)

In [ ]:
CONFIG_PATH = configs.classifiers.rusentiment_convers_bert

m = build_model(CONFIG_PATH, download=True)


2022-05-13 19:35:32.961 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/deeppavlov_data/classifiers/rusentiment_convers_bert_v0.tar.gz to /root/.deeppavlov/models/rusentiment_convers_bert_v0.tar.gz
INFO:deeppavlov.core.data.utils:Downloading from http://files.deeppavlov.ai/deeppavlov_data/classifiers/rusentiment_convers_bert_v0.tar.gz to /root/.deeppavlov/models/rusentiment_convers_bert_v0.tar.gz
100%|██████████| 657M/657M [00:24<00:00, 26.4MB/s]
2022-05-13 19:35:58.692 INFO in 'deeppavlov.core.data.utils'['utils'] at line 272: Extracting /root/.deeppavlov/models/rusentiment_convers_bert_v0.tar.gz archive into /root/.deeppavlov/models/classifiers
INFO:deeppavlov.core.data.utils:Extracting /root/.deeppavlov/models/rusentiment_convers_bert_v0.tar.gz archive into /root/.deeppavlov/models/classifiers
2022-05-13 19:36:07.471 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/deeppavlov_d

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.

2022-05-13 19:36:42.798 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/classifiers/rusentiment_convers_bert_v0/classes.dict]
INFO:deeppavlov.core.data.simple_vocab:[loading vocabulary from /root/.deeppavlov/models/classifiers/rusentiment_convers_bert_v0/classes.dict]



2022-05-13 19:36:42.941731: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-05-13 19:36:43.049782: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2022-05-13 19:36:43.050237: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x61fef800 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-05-13 19:36:43.050268: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



...


2022-05-13 19:36:57.193050: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 367248384 exceeds 10% of system memory.
2022-05-13 19:36:57.433637: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 367248384 exceeds 10% of system memory.
2022-05-13 19:36:58.138889: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 367248384 exceeds 10% of system memory.
2022-05-13 19:36:58.214634: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 367248384 exceeds 10% of system memory.
2022-05-13 19:36:58.470302: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 367248384 exceeds 10% of system memory.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


Instructions for updating:
Use standard file APIs to check for files with this prefix.
2022-05-13 19:37:05.874 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/classifiers/rusentiment_convers_bert_v0/model]
INFO:deeppavlov.core.models.tf_model:[loading model from /root/.deeppavlov/models/classifiers/rusentiment_convers_bert_v0/model]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/classifiers/rusentiment_convers_bert_v0/model


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/classifiers/rusentiment_convers_bert_v0/model


In [ ]:
def texts (val,col,col1,col3):
    text = val[col]
    old_genre = val[col1]
    plot=val[col3]

    predicted_genre = m([text])
    

    print(f'\nOld Genre:'.ljust(16,' '), f'{[old_genre]}\n')
    print(f'Predicted Genre: {predicted_genre}\n')
    print(f'Plot: {plot}\n')
    print("-------------------------------------------")

In [ ]:
i=np.random.randint(100)
texts(rev_ru.iloc[i],'text_clean','sentiment','review')


Old Genre:      ['positive']

Predicted Genre: ['neutral']

Plot: Художественный фильм 'The Master' умнейшего режиссёра и творца Пола Томаса Андерсона принадлежит к той плеяде произведений искусства, которые бросают зрителю вызов. От нас требуется максимально возможная вовлечённость в процесс, наличие хоть какого-то багажа знаний за плечами, а также, при необходимости, согласие на возвращение к работе спустя какое-то время.

В центре сюжета истории двух людей: Фредди Куэлла - демобилизованного моряка, прошедшего Вторую Мировую и ищущего место в жизни, и Ланкастера Додда - известного философа, писателя и основателя религиозного культа, идея которого заключается в том, что каждый человек живёт на протяжении триллионов лет и способен в нынешней 'фазе' стать абсолютно идеальным. 

Встреча главных героев случается в наилучшей временной точке для обоих - Фредди безнадёжно пытается выкарабкаться из первобытного состояния и найти цель в жизни, а Ланкастеру необходим 'необычный' ученик, которо

# Подготовка модели

In [ ]:
shortlisted_genres = df["Genre"].value_counts().reset_index(name="count").query("count > 200")["index"].tolist()

df = df[df["Genre"].isin(shortlisted_genres)].reset_index(drop=True)

df = df.sample(frac=1).reset_index(drop=True)

df = df.groupby("Genre").head(400).reset_index(drop=True)

label_encoder = LabelEncoder()
df["genre_encoded"] = label_encoder.fit_transform(df["Genre"].tolist())

df = df[["Plot", "Genre", "genre_encoded"]]


In [ ]:
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "max_seq_length": 100,
    "train_batch_size": 3,
    "num_train_epochs": 2,
}

model = ClassificationModel('bert', 'bert-base-cased', num_labels=len(shortlisted_genres),use_cuda=True, args=model_args)


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [ ]:
torch.cuda.empty_cache()
torch.cuda.is_available()

True

In [ ]:
train_df, eval_df = train_test_split(df, test_size=0.2, stratify=df["Genre"], random_state=42)

model.train_model(train_df[["Plot", "genre_encoded"]])

result, model_outputs, wrong_predictions = model.eval_model(eval_df[["Plot", "genre_encoded"]])
print(result)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/5102 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_100_18_2


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/1701 [00:00<?, ?it/s]

...

  0%|          | 0/1276 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_100_18_2


Running Evaluation:   0%|          | 0/160 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.42013233257640376, 'eval_loss': 1.795600265264511}


{'mcc': 0.42013233257640376, 'eval_loss': 1.795600265264511}


# Вывод модели

In [ ]:
def bert (val):
    text = val['Plot']
    old_genre = val['Genre']

    predicted_genre_encoded, raw_outputs = model.predict([text])
    predicted_genre_encoded = np.array(predicted_genre_encoded)
    predicted_genre = label_encoder.inverse_transform(predicted_genre_encoded)[0]

    print(f'\nOld Genre:'.ljust(16,' '), f'{old_genre}\n')
    print(f'Predicted Genre: {predicted_genre}\n')
    print(f'Plot: {text}\n')
    print("-------------------------------------------")

In [ ]:
i=np.random.randint(1400)
bert(gen_un.iloc[i])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


Old Genre:      unknown

Predicted Genre: adventure

Plot: The Arena, set in the ancient Roman town of Brundusium, is about a group of slave girls who are sold to a man named Timarchus (Daniele Vargas), the organizer of the events that take place in the town’s colosseum. After a fight breaks out amongst the girls, Timarchus gets the idea of putting the women in the ring to fight to the death. The recently captured Mamawi (Pam Grier) and Bodicia (Margaret Markov) realize they must stick together if they are to survive.

-------------------------------------------
